<a href="https://colab.research.google.com/github/adsLopess/Linguistica_Computacional/blob/main/lc_tokenizadores_normalizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 3 - Normalização de Texto
## DCC: Linguística Computacional

## Tokenizadores

Importando a biblioteca de expressões regulares do Python

In [ ]:
import re

### Tokenizador por Espaço

Desenvolvido com base no principal delimitador para uma grande parcela das línguas naturais humanas: o espaço

In [ ]:
texto = "No meio do caminho tinha uma pedra."

texto.split()

['No', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra.']

### Tokenizador baseado numa expressão regular

Segmenta as palavras de um texto com base em delimitadores como espaço, pontuações e início/fim de uma sequência (\b)

In [ ]:
texto = "No meio do caminho tinha uma pedra."

re.sub(r"(\b)", r" \1", texto).split()

['No', 'meio', 'do', 'caminho', 'tinha', 'uma', 'pedra', '.']

### Tokenizador baseado em Regras

1. Buscar todas as ocorrências de valores numéricos e financeiros (R$1,00; $46; etc.)

2. Buscar todas as ocorrências de sequências de 1 ou mais caracteres

3. Buscar todas as ocorrências de sequências sem espaço


In [ ]:
texto = "Eu paguei R$456,00 pelo setup. O que acha?"
regex = r"R?\$?[\d\.\,]+|\w+|\S+"
re.findall(regex, texto)

['Eu', 'paguei', 'R$456,00', 'pelo', 'setup', '.', 'O', 'que', 'acha', '?']

### Tokenizador baseado em Regras do NLTK

In [ ]:
import nltk
nltk.download('punkt')

versos = """O menino jogou bola ontem às 16:00."""

nltk.word_tokenize(versos, language='english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['O', 'menino', 'jogou', 'bola', 'ontem', 'às', '16:00', '.']

In [ ]:
text = """Hello everyone!!!"""

nltk.word_tokenize(text, language='english')

['Hello', 'everyone', '!', '!', '!']

### Byte-Pair Encoding (BPE)

[Fonte](https://huggingface.co/docs/tokenizers/python/latest/quicktour.html)

Baixando as dependências

In [ ]:
!pip install transformers

Baixando córpus do Wikipedia para treinar o tokenizador

[Fonte](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/)

In [ ]:
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
!unzip wikitext-103-raw-v1.zip

--2024-02-07 16:29:27--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.227.96, 52.216.215.32, 16.182.98.32, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.227.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191984949 (183M) [application/zip]
Saving to: ‘wikitext-103-raw-v1.zip’

wikitext-103-raw-v1 100%[===================>] 183.09M  46.3MB/s    in 4.2s    

2024-02-07 16:29:32 (43.3 MB/s) - ‘wikitext-103-raw-v1.zip’ saved [191984949/191984949]

Archive:  wikitext-103-raw-v1.zip
   creating: wikitext-103-raw/
  inflating: wikitext-103-raw/wiki.test.raw  
  inflating: wikitext-103-raw/wiki.valid.raw  
  inflating: wikitext-103-raw/wiki.train.raw  


Inicializando o tokenizador

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

Inicializando o Módulo de Treinamento

Define-se um vocabulário desejado com 30000 símbolos

In [ ]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                     vocab_size=30000,
                     min_frequency=0,
                     continuing_subword_prefix="##")

Definindo pré-tokenizador por espaço

In [ ]:
from tokenizers.pre_tokenizers import Whitespace

tokenizer.pre_tokenizer = Whitespace()

Treinando o tokenizador

In [ ]:
files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]
tokenizer.train(files, trainer)

Salvando o Tokenizador

In [ ]:
tokenizer.save("tokenizer-wiki.json")

Carregando o tokenizador

In [ ]:
tokenizer = Tokenizer.from_file("tokenizer-wiki.json")

Tokenizando textos

In [ ]:
texto = "I don't go out tonight."
output = tokenizer.encode(texto)
output.tokens

['I', 'don', "'", 't', 'go', 'out', 'ton', '##ight', '.']

In [ ]:
tokenizer.decode(output.ids)

"I don ' t go out ton ##ight ."

### Byte-level BPE

[Fonte](https://huggingface.co/blog/how-to-train)

Inicializando o tokenizador e o córpus de treinamento (Wikipedia)

In [ ]:
from tokenizers import ByteLevelBPETokenizer

files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]

tokenizer = ByteLevelBPETokenizer()

Treinando o tokenizador

In [ ]:
tokenizer.train(files=files, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

Salvando o tokenizador

In [ ]:
tokenizer.save("tokenizer-wiki.json")

Tokenizando um texto

In [ ]:
output = tokenizer.encode("Eu estou na aula de Línguística Computacional.")
output.tokens

['E',
 'u',
 'Ġest',
 'ou',
 'Ġna',
 'Ġa',
 'ula',
 'Ġde',
 'ĠL',
 'ÃŃn',
 'gu',
 'ÃŃ',
 'st',
 'ica',
 'ĠComp',
 'ut',
 'ac',
 'ional',
 '.']

### WordPiece

Inicializando o Tokenizador

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]

tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

Inicializando o módulo de treinamento

In [ ]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                     vocab_size=30000,
                     min_frequency=0,
                     continuing_subword_prefix="##")

Inicializando pré-tokenizador

In [ ]:
from tokenizers.pre_tokenizers import Whitespace

tokenizer.pre_tokenizer = Whitespace()

Treinando o tokenizador

In [ ]:
files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]
tokenizer.train(files, trainer)

Tokenizando o texto

In [ ]:
output = tokenizer.encode("Eu estou na aula.")
output.tokens

['Eu', 'est', '##ou', 'na', 'a', '##ula', '.']

### Unigram

Semelhante aos outros modelos. Pode precisar de GPU

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import Unigram
from tokenizers.trainers import UnigramTrainer

files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]

# inicializando o tokenizador
tokenizer = Tokenizer(Unigram())
# inicilizando o módulo de treinamento
trainer = UnigramTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                     vocab_size=30000,
                     min_frequency=0,
                     continuing_subword_prefix="##")
# treinando o tokenizador
files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]
tokenizer.train(files, trainer)

In [ ]:
output = tokenizer.encode("I will play games tonight.")
output.tokens

## Capitalização

Processo de colocar os tokens em letra minúscula para normalização do texto.

In [ ]:
import nltk
# nltk.download('punkt')

versos = """No meio do caminho tinha uma pedra
Tinha uma pedra no meio do caminho""".lower()

nltk.word_tokenize(versos, language='portuguese')

### Tokenização de Sentenças

In [ ]:
import nltk
# nltk.download('punkt')

texto = 'Eu estou na aula de Linguística Computacional. Os estudantes são muito bons.'

nltk.sent_tokenize(texto, language='portuguese')

['Eu estou na aula de Linguística Computacional.',
 'Os estudantes são muito bons.']

## Lematização

Instalando o Spacy

In [ ]:
!pip3 install -U pip setuptools wheel
!pip3 install -U spacy[cuda102]==3
!python3 -m spacy download pt_core_news_lg

In [ ]:
import spacy

spacy.prefer_gpu()
nlp = spacy.load("pt_core_news_lg")

In [ ]:
doc = nlp("O passado é só uma história que nos contamos.")

for token in doc:
  print(token, token.lemma_)

O O
passado passar
é ser
só só
uma umar
história história
que que
nos o
contamos contar
. .


## Radicalização

Identifica e remove prefixos e sufixos, buscando a raiz da palavra.

In [ ]:
import nltk
# nltk.download('rslp')

raiz = nltk.stem.RSLPStemmer()

tokens = nltk.word_tokenize('A comida estava gostosa', language='portuguese')
[raiz.stem(token) for token in tokens]

['a', 'com', 'est', 'gost']